# Análise comparativa de modelos
 - Conjunto de dados: `tip` (Gorjetas)
 - Cientistas de dados:
   - Carlos Stefano (carlos.stefanofilho@gmail.com)
   - Madson Dias (madsonddias@gmail.com)
   - Tayná Fiusa (taynafiuza2@gmail.com)

---

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# pipelines e transformadores
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer

# codificação de variáveis
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

# normalização
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# dados faltantes
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

# modelagem
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, RandomizedSearchCV

In [ ]:
# @title Leitura do conjunto e criação do dicionário de dados

df = pd.read_csv("https://raw.githubusercontent.com/atlantico-academy/datasets/refs/heads/main/tips.csv")

# @title Dicionário de dados
df_dict = pd.DataFrame([
    {
        "variavel": "total_bill",
        "descricao": "Total pago da conta em dólares.",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    {
        "variavel": "tip",
        "descricao": "Valor da gorjeta dada ao garçom em dólares.",
        "tipo": "quantitativa",
        "subtipo": "contínua",
    },
    {
        "variavel": "sex",
        "descricao": "Gênero do cliente (Male/Female).",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "smoker",
        "descricao": "Indica se o cliente é fumante (Yes/No).",
        "tipo": "qualitativa",
        "subtipo": "nominal",
    },
    {
        "variavel": "day",
        "descricao": "Dia da semana da refeição (Thur, Fri, Sat, Sun).",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
    {
        "variavel": "time",
        "descricao": "Período do dia em que ocorreu a refeição (Lunch/Dinner).",
        "tipo": "qualitativa",
        "subtipo": "ordinal",
    },
    {
        "variavel": "size",
        "descricao": "Número de pessoas na mesa.",
        "tipo": "quantitativa",
        "subtipo": "discreta",
    }
])
df_dict

,variavel,descricao,tipo,subtipo
0,total_bill,Total pago da conta em dólares.,quantitativa,contínua
1,tip,Valor da gorjeta dada ao garçom em dólares.,quantitativa,contínua
2,sex,Gênero do cliente (Male/Female).,qualitativa,nominal
3,smoker,Indica se o cliente é fumante (Yes/No).,qualitativa,nominal
4,day,"Dia da semana da refeição (Thur, Fri, Sat, Sun).",qualitativa,ordinal
5,time,Período do dia em que ocorreu a refeição (Lunc...,qualitativa,ordinal
6,size,Número de pessoas na mesa.,quantitativa,discreta


In [ ]:
# @title Seleção de variáveis e separação de entradas e saídas
target_variable = ['tip']
useless_variables =  (
    df_dict
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)
unused_variables = useless_variables + target_variable
nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel not in @unused_variables")
    .variavel
    .to_list()
)
discrete_variables = (
    df_dict
    .query("subtipo == 'discreta' and variavel not in @unused_variables")
    .variavel
    .to_list()
)

X = df.drop(columns=unused_variables)
y = df[target_variable]

## Preparação de dados

Cada um dos tipos de variáveis foi submetido a um fluxo de tratamento de dados específico, a saber:

### Variáveis quantitativas
 - **Contínuas**: imputação de valores faltantes através da média e normalização min-max.
 - **Discretas**: imputação de valores faltatnes através da mediana e normalização min-max.

### Variáveis qualitativas
 - **Ordinais**: imputação de valores faltantes através da moda e codificação via *one-hot encoding*.
 - **Nominais**: imputação de valores faltantes a partir do valor mais frequente e codificação ordinal de valores.

In [ ]:
# variáveis discretas
discrete_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')), # tratamento de dados faltantes
    # tratamento de dados discrepantes
    ("normalization",  MinMaxScaler())# normalização
])
# variáveis contínuas
continuous_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    # tratamento de dados discrepantes
    ("normalization",  MinMaxScaler())# normalização
])
# variáveis ordinal
ordinal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ("encoding", OrdinalEncoder(categories=[['Thur', 'Fri', 'Sat', 'Sun'], ['Lunch', 'Dinner']])) # codificação de variáveis
])
# variáveis nominais
nominal_preprocessing = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)) # , drop='if_binary'
])

## Validação cruzada

Iremos análisar quatro modelos, que serão testados utilizando a validação cruzada de monte-carlo com 30 repetições. Os modelos utilizados na análise são:

 - Regressão linear (Linear Regression)
 - K-vizinhos mais próximos (K-Nearest-Neighbors)
 - Máquinas de vetores-suporte (Support Vector Machine)
 - Árvores de decisão (Decision Tree)

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo. Tal otimização será realizada utilizando com um validação cruzada k-fold a partir dos dados de treinamento.

Utilizaremos as seguintes métricas para análise:
 - **Erro mério absoluto (*mean absolute error*, MAE)**: mede o erro médio absoluto entre valores reais e previstos. Tem interpretação direta em unidades da variável alvo.
 - **Erro médio quadrático (*mean squared error*, MSE)**: penaliza mais fortemente os grandes erros (pois eleva ao quadrado).
 - **Coeficiente de determinação (R² score)**: mede quanto o modelo consegue reduzir o erro comparado a média dos dados. Varia entre menos infinito até 1. Um bom modelo tende a ter valores próximos de 1.
 - **Erro percentual médio absoluto (*mean absolute percentage error*, MAPE)**: mede o erro percentual médio. É mais indicado quando temos valores de diferentes escalas.

In [ ]:
# @title Implementação dos modelos
preprocessing = ColumnTransformer(transformers=[
    ("ordinal", ordinal_preprocessing, ordinal_variables),
    ("nominal", nominal_preprocessing, nominal_variables),
    ("discrete", discrete_preprocessing, discrete_variables),
    ("continuous", continuous_preprocessing, continuous_variables),
], remainder='passthrough')

modelos = [
    {
        "nome": "LRG",
        "objeto": LinearRegression(),
        "hp": {}
    }, {
        "nome": "KNN",
        "objeto": KNeighborsRegressor(),
        "hp": {
            'n_neighbors': np.arange(1, 31, 5), # Número de vizinhos entre 1 e 30
            'weights': ['uniform', 'distance'], # Peso uniforme ou baseado na distância
            'p': [1, 2] # Distância de Manhattan (p=1) ou Euclidiana (p=2)
        }
    }, {
        "nome": "DTR",
        "objeto": DecisionTreeRegressor(random_state=42),
        "hp": {
            'max_depth': [None] + list(np.arange(2, 20, 4)), # Profundidade máxima
            'max_features': [None, 'sqrt', 'log2'] # Máximo de features
        }
    }, {
        "nome": "SVR",
        "objeto": SVR(),
        "hp": {
            'C': np.logspace(-3, 3, 10),     # Regularização
            'epsilon': np.logspace(-4, 0, 10),  # Insensibilidade à margem
            'kernel': ['linear', 'rbf', 'sigmoid'],  # Kernel a ser usado
        }
    }
]

In [ ]:
# @title Aplicação da validação cruzada
cv = ShuffleSplit(n_splits=30, test_size=.2, random_state=42)
metrics = {
    'neg_mean_absolute_error': 'MAE',
    'neg_mean_squared_error': 'MSE',
    'r2': 'R2',
    'neg_mean_absolute_percentage_error': 'MAPE'
}

results = []
# aplica validação cruzada em todos os modelos
for modelo in modelos:
    # aplicar random search
    random_search = RandomizedSearchCV(
        estimator=modelo["objeto"],
        param_distributions=modelo["hp"],
        n_iter=1 if modelo["nome"] == "LRG" else 10, # Número de combinações de parâmetros a testar
        scoring='neg_mean_squared_error',  # Métrica de avaliação
        cv=5, # Número de divisões para validação cruzada
        random_state=42, # Reprodutibilidade
        n_jobs=-1 # Paralelismo
    )
    # aplica validação cruzada
    approach = Pipeline(steps=[
        ("preprocessing", preprocessing),
        ("model", random_search)
    ])
    metric_results = cross_validate(approach, X=X, y=y.values.ravel(), cv=cv, scoring=list(metrics.keys()))
    # adiciona o nome do modelo ao dicionário de resultados
    metric_results['modelo'] = [modelo["nome"]] * len(metric_results['fit_time'])
    # adiciona os novos resultados a lista final de resultados
    results.append(pd.DataFrame(metric_results))
final_results = pd.concat(results, axis=0)

In [ ]:
# @title Apresentação de resultados

# função para hilight de melhores resultados
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.min(s.values), props, '')
        if s.name[0].endswith('R2'):
            return np.where(s == np.max(s.values), props, '')
        return np.where(s == np.min(s.values), props, '')

# atualização de valores das métricas
for metric in metrics.keys():
    if 'neg' in metric:
        final_results[f"test_{metric}"] *= -1


# apresentação de resultados
(
    final_results
    .rename(columns={f"test_{name}": value for name, value in metrics.items()})
    .groupby("modelo").agg(["mean", "std"]).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

## Conclusão

Em relação ao desvio padrão, o SVR, embora mais estável que a média, teve um desvio um pouco maior em R² (0.162506) quando comparado à Regressão Linear (0.115598), o que pode indicar alguma sensibilidade à divisão dos dados. Em relação a média, o SVR apresentou o melhor desempenho preditivo geral, obtendo os menores valores de erro (MAE = 0.765721, MSE = 1.085032, MAPE = 0.278470) e o maior coeficiente de determinação (R² = 0.401788), indicando maior precisão e capacidade de explicação da variabilidade da variável alvo. A regressão linear também teve desempenho competitivo, especialmente no R² (0.395362), com erros ligeiramente superiores. Já os modelos baseados em árvore (DTR) e vizinhos mais próximos (KNN) apresentaram desempenhos inferiores em todas as métricas, sugerindo menor adequação ao padrão dos dados nesse problema específico. Os desvios padrão das métricas mostram que todos os modelos apresentaram variações aceitáveis entre as execuções.